# Project: 2DEG mobility plottings - temperature variation

# 1. Settings

## 1.1 Import modules

In [59]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import colors, cm
import matplotlib.ticker as ticker

In [60]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
import sys, os
from pathlib import Path
mobilitypy_tutorial_path = Path().absolute()
sys.path.insert(0, os.path.join(mobilitypy_tutorial_path, '../'))
from mobilitypy import Plottings, PlotQuasi3DFuns

In [62]:
save_figure_dir = os.path.join(mobilitypy_tutorial_path,'../../OUTPUTS/FIGS')
savefigure = True
FigFormat = 'png'
FigFormatPaper = 'eps'
fig_save_dpi = 300
color_map_plot = 'viridis'
print_align_space = 20

In [63]:
folder_2deg_data_parent = '/home/Docs5/badal.mondal/linuxhome/MyFolder/Projects/DeviceSimulation/NextnanoSimulations/DATAs/AlGaN_HEMT/AlGaN_AlGaN_AlN_HEMT/nnp/'
folder_2deg_density_file = os.path.join(folder_2deg_data_parent, '2DEHG_density/OneDim', 'sim_post_process_data.xlsx')
folder_2deg_mobility_file = os.path.join(folder_2deg_data_parent, '2DEHG_mobility/OneDim', 'sim_post_process_data.xlsx')

## 1.2 Set physical constants

In [64]:
# n_2d => in nm^-2
# rms_roughness => nm^-1
# corr_len => nm^-1
# n_dis => nm^-2
# f_dis => unit less

In [65]:
plot_mobilities = ['IFR', 'DIS', 'AD', 'AP', 'POP', 'TOT', 'LFOM', 'LFOMnorm', '2DEG']
plots_list_individual = ['TOT', 'LFOM', 'LFOMnorm', '2DEG', 'R']
len_plots = len(plot_mobilities)
rescale_2deg_fact=1e13

x_label_text = 'Al composition barrier, y'
x_p_label_text = 'Al composition channel, x'
y_label_text = 'Barrier thickness, L$_\\mathrm{B}$(nm)'
z_label = {'General': r'2DEG mobility, $\mu$ ($\mathrm{cm}^2\mathrm{V}^{-1}\mathrm{s}^{-1}$)', 
           'TOT': r'2DEG mobility, $\mu$ ($\mathrm{cm}^2\mathrm{V}^{-1}\mathrm{s}^{-1}$)',
           'LFOM': r'LFOM (MW/cm$^2$)',
           'LFOMnorm': r'LFOM$_\mathrm{(Al,Ga)N}$/LFOM$_\mathrm{highest}$', 
           'LFOMnorm_2': r'LFOM$_\mathrm{(Al,Ga)N}$/LFOM$_\mathrm{GaN}$',
           '2DEG': r'2DEG density, n$_{\mathrm{2D}}$ ($\mathrm{10}^{13}\mathrm{cm}^{-2}$)', 
           'R': r'Sheet resistance, R$_\mathrm{sh}$ ($\Omega /\boxdot$)'}

In [66]:
params = {'axes.labelsize': 20,'axes.titlesize': 20,
          'xtick.labelsize':20,
                  'xtick.major.width':2,
                  'xtick.major.size':5,
                  'xtick.minor.width':2,
                  'xtick.minor.size':3,
                  'ytick.labelsize': 20,
                  'ytick.major.width':2,
                  'ytick.major.size':5,
                  'ytick.minor.width':2,
                  'ytick.minor.size':3,
                  'errorbar.capsize':2}
plt.rcParams.update(params)
plt.rc('font', size=20)

# 2. Read mobilities, FOMs from saved database

2DEG density negligibly depend on Temperature for both GaN- and AlGaN-channel HEMT

## 2.1 Al0.25Ga0.75N(25nm)/GaN(305nm) HEMT LFOM for normalization

In [67]:
mobility_ref_dff = pd.read_excel(folder_2deg_mobility_file, index_col=0, sheet_name='ref_Al25Ga75N_GaN') 

## 2.2 AlGaN/AlGaN mobilities and figure-of-merit

In [68]:
mobility_dff = pd.read_excel(folder_2deg_mobility_file, index_col=0, sheet_name='AlGaN_AlGaN') 

In [69]:
# Highest_LFOM = []
# for temperature, Combined_df in Combined_dff.items():
#     tmp_best = Combined_df.loc[[Combined_df['LFOM'].idxmax()]]
#     tmp_best['T'] = temperature
#     Highest_LFOM.append(tmp_best)
# highest_LFOM_df = pd.concat(Highest_LFOM)

### 2.3.6 Collect AlN/AlGaN mobility and lateral figure-of-merit

Note: Highest LFOM is found for AlN in Barrier and barrier thickness of 50 nm always

Note: AlN/AlGaN-channel/AlN normalized LFOM are normalized w.r.t Al0.25Ga0.75N(25nm)/GaN(305nm) GaN-channel LFOM at the same temperature

In [70]:
mobility_dfff = {}
for tb in [50, 25]:
    if tb == 50:
        mobility_df = mobility_dff[(mobility_dff['AlContentBarrier']>0.99) & (mobility_dff['ThicknessAlGaNBarrier']>49)].copy()
    elif tb == 25:
        mobility_df = mobility_dff[(mobility_dff['AlContentBarrier']>0.99) & (mobility_dff['ThicknessAlGaNBarrier']>24) & (mobility_dff['ThicknessAlGaNBarrier']<26)].copy()
    else:
        break
    mobility_df.reset_index(drop=True, inplace=True)
    mobility_df['2DEG'] =  mobility_df['2DEG_device']/rescale_2deg_fact
    mobility_df['2DHG'] =  mobility_df['2DHG_device']/rescale_2deg_fact
    mobility_df['LFOMnorm'] = np.nan

    mobility_dfff[tb] = mobility_df

In [71]:
plots_data = {}
for tb, mobility_df in mobility_dfff.items():
    for index, row in mobility_df.iterrows():
        ref_LFOM = mobility_ref_dff.loc[mobility_ref_dff['T']==row['T'], 'LFOM'].iloc[0]
        mobility_df.loc[index,'LFOMnorm'] = row['LFOM']/ref_LFOM

    plots_data[tb] = {}
    plots_data[tb]['comp_'] = np.array(mobility_df['AlContentChannel'], dtype=float)
    plots_data[tb]['temperature_'] = np.array(mobility_df['T'], dtype=float)
    plots_data[tb]['lfom_'] = np.array(mobility_df['LFOM'], dtype=float)
    plots_data[tb]['lfom_norm_'] = np.array(mobility_df['LFOMnorm'], dtype=float)
    tot_mu = np.array(mobility_df['TOT'], dtype=float)
    tot_mu_cp = tot_mu.copy()
    tot_mu_cp[tot_mu_cp <1] = np.nan # put to 1 below 1 cm^2V^-1s^-1 mobilities. For better plottings
    plots_data[tb]['tot_mu'] = tot_mu_cp

In [72]:
# comp_ = np.array(mobility_df['AlContentChannel'], dtype=float)
# temperature_ = np.array(mobility_df['T'], dtype=float)
# lfom_ = np.array(mobility_df['LFOM'], dtype=float)
# lfom_norm_ = np.array(mobility_df['LFOMnorm'], dtype=float)
# tot_mu = np.array(mobility_df['TOT'], dtype=float)
# tot_mu_cp = tot_mu.copy()
# tot_mu_cp[tot_mu_cp <10] = np.nan # put to nan below 10 cm^2V^-1s^-1 mobilities

In [73]:
plots_data_maxms = {}
for tb, mobility_df in mobility_dfff.items():
    XX, YY = [], [] # where norm_lfom > 1
    XXX, YYY = [], [] # where norm_lform is highest at each T
    df_group_T = mobility_df.groupby(['T'])
    for name, group in df_group_T:
        pp_ = group['LFOM'].argmax()
        XXX.append(group.iloc[pp_]['AlContentChannel'])
        YYY.append(name[0])
        tmp_ = group[group['LFOMnorm']>1]
        if len(tmp_) > 0:
            XX.append(np.array(tmp_['AlContentChannel']))
            YY.append(np.array(tmp_['T']))
    plots_data_maxms[tb] = (np.concatenate(XX), np.concatenate(YY), XXX, YYY)

## 2.4 Plot AlN/AlGaN HEMT mobilities and FOM

In [74]:
tick_multiplicator = [0.1, 0.05, 200, 100]

In [75]:
lpltq3d = PlotQuasi3DFuns(save_figure_dir=f'{save_figure_dir}/2DEG_mobilities/Temperature')

In [19]:
for tb in mobility_dfff:
    comp_,temperature_,tot_mu = plots_data[tb]['comp_'], plots_data[tb]['temperature_'], plots_data[tb]['tot_mu']
    fig, ax = plt.subplots(figsize=(8,5.5), constrained_layout=True)
    fig, _ = lpltq3d.Plotq3D(comp_,temperature_,tot_mu, fig=fig, ax=ax,
                             xmin=0.475, xmax=0.975, ymin=-40, ymax=850,
                             x_label=x_p_label_text, y_label= 'T (K)',
                             tick_multiplicator=tick_multiplicator,
                             show_contour_lines=False, marker='s', marker_size=38**2,
                             cbar_text=z_label['TOT'],show_colorbar=True,
                             plot_controur=0, plot_scatter=1, show_plot=False)
    lpltq3d.save_figure(f'Highest_FOM_mobility_Tb{tb}.{FigFormat}', savefig=savefigure, show_plot=True,
                        fig=fig, CountFig=None, dpi=fig_save_dpi)

In [20]:
# Plot mobilities in log scale
for tb in mobility_dfff:
    comp_,temperature_,tot_mu = plots_data[tb]['comp_'], plots_data[tb]['temperature_'], plots_data[tb]['tot_mu']
    fig, ax = plt.subplots(figsize=(8,5.5), constrained_layout=True)
    vmin = np.nanmin(tot_mu)
    vmax = np.nanmax(tot_mu)
    norm = colors.LogNorm(vmin=vmin, vmax=vmax) 
    cbar_mapable = cm.ScalarMappable(norm=norm, cmap='viridis')
    fig, _ = lpltq3d.Plotq3D(comp_,temperature_,tot_mu, fig=fig, ax=ax,
                             xmin=0.475, xmax=0.975, ymin=-40, ymax=850,
                             x_label=x_p_label_text, y_label= 'T (K)',
                             tick_multiplicator=tick_multiplicator,cbar_mappable=cbar_mapable,
                             norm=norm, vmin=vmin, vmax=vmax,
                             show_contour_lines=False, marker='s', marker_size=38**2,
                             cbar_text=z_label['TOT'],show_colorbar=True,
                             plot_controur=0, plot_scatter=1, show_plot=False)
    lpltq3d.save_figure(f'Highest_FOM_log_mobility_Tb{tb}.{FigFormat}', savefig=savefigure, show_plot=True,
                        fig=fig, CountFig=None, dpi=fig_save_dpi)

In [22]:
for tb in mobility_dfff:
    comp_,temperature_,lfom_norm_ = plots_data[tb]['comp_'], plots_data[tb]['temperature_'], plots_data[tb]['lfom_norm_']
    XX, YY, XXX, YYY = plots_data_maxms[tb]
    fig, ax = plt.subplots(figsize=(7.6,5.5), constrained_layout=True)
    fig, _ = lpltq3d.Plotq3D(comp_,temperature_,lfom_norm_, fig=fig, ax=ax,
                             xmin=0.475, xmax=0.975, ymin=-40, ymax=850,
                             x_label=x_p_label_text, y_label= 'T (K)',
                             tick_multiplicator=tick_multiplicator,
                             show_contour_lines=False, marker='s', marker_size=38**2,
                             cbar_text=z_label['LFOMnorm_2'],show_colorbar=True,
                             plot_controur=0, plot_scatter=1, show_plot=False)
    ax.scatter(XX, YY, marker='x', s=20**2, c='k')
    ax.scatter(XXX, YYY, marker='o', s=10**2, c='k')
    lpltq3d.save_figure(f'Highest_FOM_norm_Tb{tb}.{FigFormat}', savefig=savefigure, show_plot=True,
                        fig=fig, CountFig=None, dpi=fig_save_dpi)

In [91]:
markers_ = ['^', '*', 'd', 'o', 's', 'p']*3
markers_ = ['.','>','D','o','*','h','p','s','^', '*', 'd', 'o', 's', 'p']*3
colors_ = ['black','darkred', 'indianred','red', 'chocolate', 'peru', 'orange', 'gold', 'yellow']
for tb, mobility_df in mobility_dfff.items():
    ii = 0
    df_group_T = mobility_df.groupby(['T'])
    fig, ax = plt.subplots(constrained_layout=True)
    for name, group in df_group_T:
        #if name[0] in [100,200,300,400,600,800]:
        markeredgecolor='darkgoldenrod' if name[0]>400 else 'none'
        ax.plot(group['AlContentChannel'], group['LFOMnorm'], '-',marker=markers_[ii],
                color=colors_[ii],label=f'T={name[0]} K', ms=12, mec=markeredgecolor,mew=0.5)
        ii+=1
    #ax.set_yscale('log')
    ax.set_xlim(0.5,0.9)
    ax.set_xlabel(x_p_label_text)
    ax.set_ylabel(z_label['LFOMnorm_2'])
    #ax.legend(loc='center left', bbox_to_anchor=(1, 0.5)) 
    ax.xaxis.set_major_locator(ticker.MultipleLocator(0.1))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.05))
    #ax.yaxis.set_major_locator(ticker.MultipleLocator(0.2))
    #ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.1))
    ax.axhline(y=1,c='k', ls='--')
    lpltq3d.save_figure(f'Highest_FOM_norm_line_Tb{tb}.{FigFormatPaper}', savefig=savefigure, show_plot=True,
                        fig=fig, CountFig=None, dpi=fig_save_dpi)

In [92]:
save_file_name_ = f'temperature_legends.{FigFormatPaper}'
# Now create a new image with legends only
# adjust the figure size as necessary
fig_leg = plt.figure(figsize=(1,1))
ax_leg = fig_leg.add_subplot(111)
# add the legend from the previous axes
ax_leg.legend(*ax.get_legend_handles_labels(), ncol=3, loc='center')
# hide the axes frame and the x/y labels
ax_leg.axis('off')
#ax_leg.set_frame_on(False)
lpltq3d.save_figure(save_file_name_, fig=fig_leg, savefig=savefigure, dpi=fig_save_dpi, show_plot=False)

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [26]:
for tb, mobility_df in mobility_dfff.items():
    ii=0
    df_group_T = mobility_df.groupby(['T'])
    fig, ax = plt.subplots(figsize=(9.5,6), constrained_layout=True)
    ax2 = ax.twinx()
    for name, group in df_group_T:
        #if name[0] in [100,200,300,400,600,800]:
        ax2.plot(group['AlContentChannel'], group['LFOM'], '-',marker=markers_[ii], label=name[0], ms=12)
        ref_LFOM = mobility_ref_dff.loc[mobility_ref_dff['T']==name[0], 'LFOM'].iloc[0]
        ax.scatter(0.47,ref_LFOM, marker=markers_[ii], s=100)
        ii+=1
    ax.set_yscale('log')
    ax.set_xlabel(x_p_label_text)
    ax.set_ylabel(z_label['LFOM'])
    
    ax.xaxis.set_major_locator(ticker.MultipleLocator(0.1))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.05))
    
    ax.axvline(x=0.5, c='k', ls='--')
    #ax2.scatter([0.48]*len(lfom_r),lfom_r, marker='s')
    ax2.set_yscale('log')
    if tb == 50: 
        yminn = 7e3
    elif tb==25:
        yminn = 2e3
    ax2.set_ylim(ymin=yminn, ymax=5e4)
    #ax2.yaxis.set_major_locator(ticker.LogLocator(numticks=2))
    #ax2.yaxis.set_minor_formatter(ticker.NullFormatter())
    #ax2.minorticks_off()
    
    tmp = ax.get_xticks()
    tmp_txt = [f'{xx:.1f}' for xx in tmp]
    ax.set_xticks(list(tmp)+[0.47], tmp_txt+['0'])
    ax.set_xlim(0.45,0.9)
    lpltq3d.save_figure(f'Highest_FOM_line_Tb{tb}.{FigFormatPaper}', savefig=savefigure, show_plot=True,
                        fig=fig, CountFig=None, dpi=fig_save_dpi)

In [27]:
for tb, mobility_df in mobility_dfff.items():
    ii=0
    df_group_T = mobility_df.groupby(['T'])
    fig, ax = plt.subplots(figsize=(9,6), constrained_layout=True)
    ax2 = ax.twinx()
    for name, group in df_group_T:
        ax2.plot(group['AlContentChannel'], group['LFOM'], '-',marker=markers_[ii], label=name[0], ms=12)
        ref_LFOM = mobility_ref_dff.loc[mobility_ref_dff['T']==name[0], 'LFOM'].iloc[0]
        ax.scatter(0.47,ref_LFOM, marker=markers_[ii],s=100)
        ii+=1
    ax.set_yscale('log')
    ax.set_xlabel(x_p_label_text)
    ax.set_ylabel(z_label['LFOM'])
    
    #ax.legend(loc='center left', bbox_to_anchor=(1, 0.5)) #loc='upper center', bbox_to_anchor=(0.5, 1.3), ncol=3)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(0.1))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.05))
    
    ax.axvline(x=0.5, c='k', ls='--')
    ax2.set_yscale('log')
    ax2.set_ylim(ax.get_ylim())
    tmp = ax.get_xticks()
    tmp_txt = [f'{xx:.1f}' for xx in tmp]
    ax.set_xticks(list(tmp)+[0.47], tmp_txt+['0'])
    ax.set_xlim(0.45,0.9)
    lpltq3d.save_figure(f'Highest_FOM_line_zoom_Tb{tb}.{FigFormat}', savefig=savefigure, show_plot=True,
                        fig=fig, CountFig=None, dpi=fig_save_dpi)

In [28]:
for tb, mobility_df in mobility_dfff.items():
    XX, YY, XXX, YYY = plots_data_maxms[tb]
    fig, ax = plt.subplots(constrained_layout=True)
    ax.set_xlabel('T (K)')
    ax.set_ylabel('Al composition channel, x') #Highest LFOM (MW/cm$^2$)
    ax.plot(YYY, XXX, 'ko-', ms=12)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(200))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(100))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(0.05))
    ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.025))
    ax.set_xlim(0,800)
    lpltq3d.save_figure(f'Highest_FOM_T_comp_Tb{tb}.{FigFormatPaper}', savefig=savefigure, show_plot=True,
                        fig=fig, CountFig=None, dpi=fig_save_dpi)

In [29]:
for tb in mobility_dfff:
    comp_,temperature_,lfom_ = plots_data[tb]['comp_'], plots_data[tb]['temperature_'], plots_data[tb]['lfom_']
    XX, YY, XXX, YYY = plots_data_maxms[tb]
    vmin = np.nanmin(lfom_) #lfom_[lfom_>1].min()
    vmax = np.nanmax(lfom_)
    norm = colors.LogNorm(vmin=vmin, vmax=vmax) 
    cbar_mapable = cm.ScalarMappable(norm=norm, cmap='viridis')
    
    fig, ax = plt.subplots(figsize=(8,4.5), constrained_layout=True)
    
    fig, _ = lpltq3d.Plotq3D(comp_,temperature_,lfom_, fig=fig, ax=ax,
                             xmin=0.475, xmax=0.975, ymin=-40, ymax=850,
                             vmin=vmin, vmax=vmax,cbar_mappable=cbar_mapable,
                             x_label=x_p_label_text, y_label= 'T (K)',
                             tick_multiplicator=tick_multiplicator, norm='log',
                             show_contour_lines=False, marker='s', marker_size=38**2,
                             cbar_text=z_label['LFOM'],show_colorbar=True,
                             plot_controur=0, plot_scatter=1, show_plot=False)
    ax.scatter(XX, YY, marker='x', s=20**2, c='k')
    ax.scatter(XXX, YYY, marker='o', s=10**2, c='k')
    lpltq3d.save_figure(f'Highest_FOM_Tb{tb}.{FigFormat}', savefig=savefigure, show_plot=True,
                        fig=fig, CountFig=None, dpi=fig_save_dpi)